# Atualização de CEPs no banco licença

O Presente script tem como objetivo atualizar os CEPs da base de dados com base 
    em arquivos disponibilizados da base de dados dos Correios 

  ------------------------------
  #### Etapas                      
  ------------------------------
  - Criar a conexão com o banco de dados IBM DB2 
  - Realizar a verificação do arquivo com os CEPs
  - Realizar a comparação do que já existe no banco de dados com o que existe no arquivo
  - Atualizar dados existentes 
  - Inserir registros novos 

## Importando as bibliotecas
*********


In [35]:

#Importar as bibliotecas necessárias.
import pandas as pd
import os 
import conection.connect_db as conn_db  
import ibm_db_dbi as dbi
import numpy as np
import traceback 

## Testar conexão com o banco de dados. 

In [36]:
conn = conn_db.conexao_db()
if conn:
    pconn = dbi.Connection(conn)
    print("Conexão DBI estabelecida com sucesso.")
else:
    print("Falha ao estabelecer conexão DBI.")  
cursor = pconn.cursor()   

Conexao estabelecida com sucesso.
Conexão DBI estabelecida com sucesso.


## Ler arquivos que contem os dados novos
### Será necessário neste momento realizar a equalização dos dados, mapear conforme os arquivos

Os arquivos que serão utilizados são os de: Dados\eDNE_Basico20101\Delimitado
LOG_LOCALIDADE - Arquivo que tem as cidades, ceps e ibge
LOG_LOGRADOURO - Arquivo que tem as informações dos logradouros relacionadas aos CEPs e as cidades
LOG_BAIRRO - informação do bairro
LOG_VAR_LOG - fica o tipo do logradouro



In [ ]:

#ler arquivos, adicionar no dataframe e deixar maiusculo

#cidade 
file_cidade = r"Dados\eDNE_Basico20101\Delimitado\LOG_LOCALIDADE.TXT"
cidade_novo = pd.read_csv(file_cidade, sep="@", dtype=str, encoding='latin1', header=None, names=['LOC_NU', 'UF', 'CIDADE','CEP', 'SITUACAO', 'LOC_IN_TIPO_LOC','LOC_NU_SUB','LOC_NO_ABREV','COD_IBGE']) 
cidade_novo = cidade_novo.applymap(lambda x: x.upper() if isinstance(x, str) else x)


#ler logradouros 
import glob #glob é uma biblioteca para manipulação de arquivos que foi utilizada aqui para ler vários arquivos de logradouros de uma unica vez e adicionar ao dataframe
file_logradouros = glob.glob(r'Dados\eDNE_Basico20101\Delimitado\LOG_LOGRADOURO*.TXT')
logradouros_list = [] #lista para armazenar os dataframes de cada arquivo lido

# Ler cada arquivo e adicionar à lista
for file in file_logradouros:
    df_log = pd.read_csv(file, sep="@", dtype=str, encoding='latin1', header=None, 
                         names=['LOG_NU', 'UFE_SG','LOC_NU','BAI_NU_INI','BAI_NU_FIM',
                                 'LOG_NO', 'LOG_COMPLEMENTO', 'CEP','TIPO_LOG','LOG_STA_TLO', 
                                 'LOG_NO_ABREV'])
    logradouros_list.append(df_log) #append é usado para concatenar os dataframes na lista

logradouros = pd.concat(logradouros_list, ignore_index=True)
logradouros = logradouros.applymap(lambda x: x.upper() if isinstance(x, str) else x)


#ler complemento de logradouro
file_var_log = r"Dados\eDNE_Basico20101\Delimitado\LOG_VAR_LOG.TXT"
var_log = pd.read_csv(file_var_log, sep="@", dtype=str, encoding='latin1', header=None, names=['LOG_NU','VLO_NU','TIPO_LOG','NOME_VAR'])
var_log = var_log.applymap(lambda x: x.upper() if isinstance(x, str) else x)



#ler bairros
file_bairro = r"Dados\eDNE_Basico20101\Delimitado\LOG_BAIRRO.TXT"
bairros = pd.read_csv(file_bairro, sep="@", dtype=str, encoding='latin1', header=None, names=['BAI_NU', 'UFE_SG','LOC_NU','BAI_NO','BAI_NO_ABREV'])
bairros = bairros.applymap(lambda x: x.upper() if isinstance(x, str) else x)



C:\Users\rafael.maciel\AppData\Local\Temp\ipykernel_26588\3088210890.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cidade_novo = cidade_novo.applymap(lambda x: x.upper() if isinstance(x, str) else x)
C:\Users\rafael.maciel\AppData\Local\Temp\ipykernel_26588\3088210890.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  logradouros = logradouros.applymap(lambda x: x.upper() if isinstance(x, str) else x)
C:\Users\rafael.maciel\AppData\Local\Temp\ipykernel_26588\3088210890.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  var_log = var_log.applymap(lambda x: x.upper() if isinstance(x, str) else x)
C:\Users\rafael.maciel\AppData\Local\Temp\ipykernel_26588\3088210890.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  bairros = bairros.applymap(lambda x: x.upper() if isinstance(x, str) else x)


In [38]:
#mostrar as tabelas lidas
cidade_novo.head(10)

,LOC_NU,UF,CIDADE,CEP,SITUACAO,LOC_IN_TIPO_LOC,LOC_NU_SUB,LOC_NO_ABREV,COD_IBGE
0,1,AC,ACRELÂNDIA,69945000,0,M,NaN,ACRELÂNDIA,1200013
1,11059,AC,CAMPINAS,69929000,0,D,13,CAMPINAS,NaN
2,3,AC,BRASILÉIA,69932000,0,M,NaN,BRASILÉIA,1200104
3,4,AC,BUJARI,69926000,3,M,NaN,BUJARI,1200138
4,5,AC,CAPIXABA,69931000,0,M,NaN,CAPIXABA,1200179
5,6,AC,CRUZEIRO DO SUL,69980000,0,M,NaN,CRUZEIRO SUL,1200203
6,7,AC,EPITACIOLÂNDIA,69934000,0,M,NaN,EPITACIOLÂNDIA,1200252
7,8,AC,FEIJÓ,69960000,0,M,NaN,FEIJÓ,1200302
8,9,AC,JORDÃO,69975000,0,M,NaN,JORDÃO,1200328
9,10,AC,MÂNCIO LIMA,69990000,0,M,NaN,MÂNCIO LIMA,1200336


In [39]:
# logarouros
logradouros.head(10)

,LOG_NU,UFE_SG,LOC_NU,BAI_NU_INI,BAI_NU_FIM,LOG_NO,LOG_COMPLEMENTO,CEP,TIPO_LOG,LOG_STA_TLO,LOG_NO_ABREV
0,1,AC,16,47,NaN,NELSON MESQUITA,NaN,69918703,RUA,S,R NELSON MESQUITA
1,1001866,AC,16,55447,NaN,24 DE DEZEMBRO,NaN,69918142,RUA,S,R 24 DE DEZEMBRO
2,1004886,AC,16,32,NaN,MANOEL CEZÁRIO,NaN,69900816,TRAVESSA,S,TV MANOEL CEZÁRIO
3,1004887,AC,16,55437,NaN,SÃO JOSÉ,NaN,69915361,TRAVESSA,S,TV S JOSÉ
4,1004888,AC,16,16,NaN,COLOMBO,NaN,69905027,BECO,S,BC COLOMBO
5,1004889,AC,16,30,NaN,JOSÉ PINHO,NaN,69915536,RUA,S,R JOSÉ PINHO
6,1004890,AC,16,30,NaN,FÁTIMA MAIA,NaN,69915572,RUA,S,R FÁTIMA MAIA
7,1004891,AC,16,55480,NaN,HORTENCIA DA SILVA,NaN,69922227,RUA,S,R HORTENCIA DA SILVA
8,1004892,AC,16,55480,NaN,TUFI,NaN,69922250,RUA,S,R TUFI
9,1004893,AC,16,55480,NaN,FLOR DE JARDIM,NaN,69922253,RUA,S,R FLOR DE JD


In [40]:
# bairros
bairros.head(10)

,BAI_NU,UFE_SG,LOC_NU,BAI_NO,BAI_NO_ABREV
0,55389,AC,16,CASA NOVA,CASA NOVA
1,55390,AC,16,CONJUNTO GUIOMARD SANTOS,CJ GUIOMARD SANTOS
2,55391,AC,16,CONJUNTO OSCAR PASSOS,CJ OSCAR PASSOS
3,55392,AC,16,VITÓRIA,VITÓRIA
4,55393,AC,16,LOTEAMENTO JARDIM SÃO FRANCISCO,LOT JD S FRANCISCO
5,55394,AC,16,ELDORADO,ELDORADO
6,55395,AC,16,CHICO MENDES,CHICO MENDES
7,55396,AC,16,WANDERLEY DANTAS,WANDERLEY DANTAS
8,55397,AC,16,XAVIER MAIA,XAVIER MAIA
9,55398,AC,16,PARQUE DOS SABIÁS,PRQ SABIÁS


In [41]:
#complemento de logradouro
var_log.head(10)

,LOG_NU,VLO_NU,TIPO_LOG,NOME_VAR
0,15717,1,RUA,RUA VÊNUS
1,15722,1,CAMINHO,CAMINHO 20
2,15723,1,CAMINHO,CAMINHO 20
3,15172,1,RUA,RUA A
4,14848,1,TRAVESSA,TRAVESSA ALBERTO COIMBRA
5,14855,1,RUA,RUA ANTONIO BALBINO
6,14856,1,PRAÇA,PRAÇA ANTÔNIO BALBINO
7,14864,1,RUA,RUA ARAME
8,14869,1,RUA,RUA AUGUSTO RIBEIRO
9,14870,1,PRAÇA,PRAÇA AUGUSTO TORRES


In [42]:
#busca no banco de dados a tabela de cidade

cursor.execute("SELECT * FROM DBA.CIDADES_IBGE")
cidade = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])



In [43]:
#listar a tabela
cidade.head(10)



,IDCIDADE,UF,DESCRCIDADE,IDCEP,DTALTERACAO,CODIGOIBGE,CODIGOMUNICIPAL,UTC,SITUACAOLOGRADOURO,TIPOLOCALIDADE
0,98833,TO,PE DA SERRA,77839400.0,2020-07-08 16:56:54.625,NaN,None,-3.0,N,M
1,98841,MG,CITROLANDIA,32800000.0,2020-07-08 16:56:54.625,NaN,None,-3.0,N,M
2,98850,MG,VALADARES,36101000.0,2020-07-08 16:56:54.625,NaN,None,-3.0,N,M
3,98868,MG,CAETÉ(JUIZ DE FORA),36102000.0,2020-07-08 16:56:54.625,NaN,None,-3.0,N,M
4,98876,MG,PAULA LIMA,36103000.0,2020-07-08 16:56:54.625,NaN,None,-3.0,N,M
5,98884,MG,DELTA,38108000.0,2020-07-08 16:56:54.625,3121258.0,None,-3.0,N,M
6,98892,MT,COXIPO ACU,78101000.0,2020-07-08 16:56:54.625,NaN,None,-4.0,N,M
7,98906,MT,MACHADO,78103000.0,2020-07-08 16:56:54.625,NaN,None,-4.0,N,M
8,98914,MT,MATA DENTRO,78107000.0,2020-07-08 16:56:54.625,NaN,None,-4.0,N,M
9,98922,MT,AGUACU,78108000.0,2020-07-08 16:56:54.625,NaN,None,-4.0,N,M


In [44]:
cursor.execute("SELECT * FROM DBA.CEP_ECT")
cep = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

In [45]:
cep.head(10)

,IDCEP,IDCIDADE,BAIRRO,LOGRADOURO,COMPLEMENTO,TIPOLOGRADOURO,IDTIPOLOGRADOURO,DTALTERACAO,DESCRTIPOLOGRADOURO,IDREGIAO
0,57075773,809,SANTOS DUMONT,Rua E,(Lot Floresta),None,21,2014-06-09 03:08:25.448,Quadra,None
1,57075039,809,SANTOS DUMONT,Rua D,(Lot Primavera),None,21,2014-06-09 03:08:25.448,Quadra,None
2,57075175,809,SANTOS DUMONT,Rua B,(Cj Sta Rita),None,2,2014-06-09 03:08:25.448,Rua,None
3,57083031,809,ANTARES,Rua E,(Cj Henrique Equelman),None,2,2014-06-09 03:08:25.448,Rua,None
4,57048182,809,ANTARES,Rua R,(Lot Casa Forte),None,2,2014-06-09 03:08:25.448,Rua,None
5,57083176,809,ANTARES,Rua F,(Lot Pouso da Garça 4),None,21,2014-06-09 03:08:25.448,Quadra,None
6,57083074,809,ANTARES,Rua D,(Cj Henrique Equelman),None,2,2014-06-09 03:08:25.448,Rua,None
7,57086228,809,BENEDITO BENTES,Rua P,(Cj Depa Selma Bandeira),None,21,2014-06-09 03:08:25.448,Quadra,None
8,57086436,809,BENEDITO BENTES,Rua E,(Cj Cidade Sorriso II),None,21,2014-06-09 03:08:25.448,Quadra,None
9,57086296,809,BENEDITO BENTES,Rua 8,(Cj Paulo Bandeira),None,21,2014-06-09 03:08:25.448,Quadra,None


In [46]:
logradouros.head(10)    

,LOG_NU,UFE_SG,LOC_NU,BAI_NU_INI,BAI_NU_FIM,LOG_NO,LOG_COMPLEMENTO,CEP,TIPO_LOG,LOG_STA_TLO,LOG_NO_ABREV
0,1,AC,16,47,NaN,NELSON MESQUITA,NaN,69918703,RUA,S,R NELSON MESQUITA
1,1001866,AC,16,55447,NaN,24 DE DEZEMBRO,NaN,69918142,RUA,S,R 24 DE DEZEMBRO
2,1004886,AC,16,32,NaN,MANOEL CEZÁRIO,NaN,69900816,TRAVESSA,S,TV MANOEL CEZÁRIO
3,1004887,AC,16,55437,NaN,SÃO JOSÉ,NaN,69915361,TRAVESSA,S,TV S JOSÉ
4,1004888,AC,16,16,NaN,COLOMBO,NaN,69905027,BECO,S,BC COLOMBO
5,1004889,AC,16,30,NaN,JOSÉ PINHO,NaN,69915536,RUA,S,R JOSÉ PINHO
6,1004890,AC,16,30,NaN,FÁTIMA MAIA,NaN,69915572,RUA,S,R FÁTIMA MAIA
7,1004891,AC,16,55480,NaN,HORTENCIA DA SILVA,NaN,69922227,RUA,S,R HORTENCIA DA SILVA
8,1004892,AC,16,55480,NaN,TUFI,NaN,69922250,RUA,S,R TUFI
9,1004893,AC,16,55480,NaN,FLOR DE JARDIM,NaN,69922253,RUA,S,R FLOR DE JD


In [70]:
#fazer um join com os dataframes dos arquivos 
cidades_completas = pd.merge(cidade_novo[['LOC_NU', 'UF','CIDADE','CEP','SITUACAO','COD_IBGE','LOC_IN_TIPO_LOC']], 
                             bairros[['BAI_NU', 'LOC_NU','BAI_NO']], 
                             on='LOC_NU', 
                             how='left')

cidades_completas = pd.merge(cidades_completas, 
                             logradouros[['LOG_NU', 'LOC_NU','LOG_NO','LOG_COMPLEMENTO','CEP','TIPO_LOG','BAI_NU_INI']], 
                             left_on=['LOC_NU','BAI_NU'],right_on=['LOC_NU','BAI_NU_INI'],
                             how='left', suffixes=('', '_LOG'))

#verificação Nan, remover as linhas que só tem NaN em todas as colunas de logradouro
cidades_completas_cleaned = cidades_completas.dropna(how='all')

#substituir os NaN por vazio
cidades_completas_cleaned = cidades_completas_cleaned.fillna('')


## Iniciando as comparações para verificar o que ainda não existe na base de dados.
************

Nesta seção será feito a validação do que é cidade e o que é logradouro, se a cidade ja existe e se está como logradou ou não.



In [ ]:
#verificar as cidades que existem no arquivo mas não existem no banco de dados
cidades_arquivo = set(cidades_completas_cleaned['COD_IBGE'].unique()) # 
cidades_banco = set(cidade['COD_IBGE'].unique())

cidades_faltando = cidades_arquivo - cidades_banco